# 测试F1分数

In [1]:
import json
import requests
from typing import Dict, Any

## 加载JSON文件

In [2]:
def get_prediction_and_reference_frm_file(filename):
    result_l = json.load(open(filename, "r"))
    predictions = [result["processed_predict"] for result in result_l]
    references  = [result["processed_s_label"] for result in result_l]
    
    return (predictions, references)

In [3]:
filename = '/home/protago/jxqi/project/text2sparql/picard/train/train_0725/predictions_eval_1012.3386243386243.json'

predictions, references = get_prediction_and_reference_frm_file(filename)

In [4]:
len(predictions)

6046

# 进行查询

## 定义查询的工具函数

In [5]:
def empty(r):
    if not r:
        return True
    if 'boolean' not in r:
        if 'results' in r:
            if 'bindings' in r['results']:
                if not r['results']['bindings']:
                    return True
                if {} in r['results']['bindings']:
                    return True
    return False

In [6]:
def hitkg(query, url="http://localhost:8892/sparql", typeq="target"):
    url = "https://wikidata.demo.openlinksw.com/sparql"
    try:
        #print(query)
        proxies = { "http": None, "https": None}
        r = requests.get(url, params={'format': 'json', 'query': query}, proxies =proxies)
        json_format = r.json()
        #print(json_format)
        results = json_format
        if empty(results) and typeq == 'target':
            # print("Empty")
            return "Empty"
        else:
            # print("find result")
            #sys.exit(1)
            return results
    except Exception as err:
        #print(err)
        if typeq == 'target':
            print("no response in target query")
            #sys.exit(1)
        return ''

In [7]:
def parse_answer_from_result(result_dict):
    if 'boolean' in result_dict:
        return result_dict['boolean']
    elif 'results' in result_dict:
        raw_results = result_dict['results']['bindings']
        result = [item[result_dict['head']['vars'][0]]['value'] for item in raw_results]
        return result
    else:
        return None

In [8]:
def compute_prf1_one(prediction, reference):
    gold_result = hitkg(reference)
    gold_answer = parse_answer_from_result(gold_result)
    pred_result = hitkg(prediction)
    pred_answer = parse_answer_from_result(pred_result)
    if gold_answer is None:
        if pred_answer is None:
            recall, precision, f1 = 1, 1, 1
        else:
            recall, precision, f1 = 0, 0, 0
    elif isinstance(gold_answer, bool):
        if gold_answer == pred_answer:
            recall, precision, f1 = 1, 1, 1
        else:
            recall, precision, f1 = 0, 0, 0
    elif isinstance(gold_answer, list):
        if pred_answer is None:
            recall, precision, f1 = 0, 0, 0
        else:
            pred_answer = set(pred_answer)
            gold_answer = set(gold_answer)
            if len(pred_answer.intersection(gold_answer)) != 0:
                precision = len(pred_answer.intersection(gold_answer)) / len(pred_answer)
                recall = len(pred_answer.intersection(gold_answer)) / len(gold_answer)
                f1 = (2 * recall * precision / (recall + precision))
            else:
                recall, precision, f1 = 0, 0, 0
                
    result_dict = {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }
    
    return result_dict

In [9]:
def compute_f1_metric(predictions, references) -> Dict[str, Any]:
    from tqdm import tqdm
    precision, recall, f1= 0., 0., 0.
    total = 0
    for pred, ref in tqdm(zip(predictions, references)):
        prediction = pred
        reference = ref

        result_dict = compute_prf1_one(prediction, reference)
        p, r, f= result_dict["precision"], result_dict["recall"], result_dict["f1"]
        precision += p
        recall += r
        f1 += f

        total += 1

    return {
        "precision": float(precision/total),
        "recall": float(recall/total),
        "f1": float(f1/total)
    }

## 进行查询

In [10]:
result = compute_f1_metric(predictions, references)

1it [00:00,  1.18it/s]

no response in target query


2it [00:01,  1.23it/s]

no response in target query


4it [00:03,  1.21it/s]

...

6039it [1:25:49,  1.21it/s]

no response in target query


6040it [1:25:50,  1.21it/s]

no response in target query


6046it [1:25:55,  1.17it/s]


In [11]:
result

{'precision': 0.9349038325220925,
 'recall': 0.9347833278200463,
 'f1': 0.9347765075037738}

In [12]:
9088/128*11.5925925926

823.0740740746

In [13]:
9472/849.42

11.151138423865698

In [14]:
58.169312 - 46.507936

11.661375999999997